# 🎬 视频自动配音流水线
## Video Auto-Dubbing Pipeline

---

### 流程图 / Pipeline

```
原始视频 (MP4)
    │
    ▼
[Step 1] 环境初始化
    │
    ▼
[Step 2] Whisper 语音识别 → 转录文本
    │
    ▼
[Step 3] 生成 SRT 字幕文件
    │
    ├──► [Step 4] (可选) 嵌入字幕视频 → *_sub.mp4
    │
    ▼
[Step 5] Edge-TTS 配音 + 时长对齐 → *_dub.wav
    │
    ▼
[Step 6] 替换音频 → *_dubbed.mp4
    │
    ├──► [Step 7] (可选) 嵌入字幕视频 → *_subdubbed.mp4（最终输出）
```

### 使用说明 / Usage

1. 修改 **Cell 1（超参数配置区）** 中的路径和参数
2. 依次执行各 Cell（或 "Run All Cells"）
3. 最终输出为 `*_subdubbed.mp4`


In [ ]:
# ===== 超参数配置区（仅需修改此 Cell）=====
from pathlib import Path
import logging

# --- 输入路径 ---
VIDEO_PATH = Path("/home/tianqi/D/01_Projects/06_ffmpeg/data/IMG_9709.MOV")

# --- 输出目录（默认与视频同目录）---
OUTPUT_DIR = VIDEO_PATH.parent

# --- Whisper 转录配置 ---
WHISPER_MODEL    = "medium"   # 可选: tiny / base / small / medium / large / turbo
WHISPER_LANGUAGE = "en"      # 转录语言: "zh"=中文, "en"=英文
WHISPER_FP16     = True     # GPU 下设 True；CPU 设 False

# --- TTS 引擎选择 ---
TTS_ENGINE = "f5"   # "edge" = edge-tts（无需 GPU）| "f5" = F5-TTS Voice Cloning（需 GPU）

# --- TTS 配音配置（edge-tts）---
TTS_VOICE = "en-US-GuyNeural"
# 中文常用: zh-CN-XiaoxiaoNeural, zh-CN-YunxiNeural
# 英文常用: en-US-JennyNeural, en-US-GuyNeural

# --- F5-TTS Voice Cloning 配置（TTS_ENGINE="f5" 时生效）---
VOICE_CLONE_REF_AUDIO = Path("/home/tianqi/D/01_Projects/06_ffmpeg/data_demo/tianqiYao_voice.m4a")   # 参考音频（5~30 秒）
VOICE_CLONE_REF_TEXT  = "参考音频的对应文字内容. This is Tianqi Yao."     # 参考音频的转录文本（必填）

# --- 可选步骤开关 ---
GENERATE_SUBTITLE_VIDEO = True   # 是否生成带字幕视频 (Cell 9)

# --- 日志级别 ---
LOG_LEVEL = logging.INFO

# ===== 派生路径（自动生成，无需手动修改）=====
SRT_PATH          = OUTPUT_DIR / ("01_" + VIDEO_PATH.stem + ".srt")
TTS_TMP_DIR       = OUTPUT_DIR / "_tts_tmp"
DUB_WAV_PATH      = OUTPUT_DIR / ("02_" + VIDEO_PATH.stem + "_dub.wav")
SUB_VIDEO_PATH    = OUTPUT_DIR / ("03_" + VIDEO_PATH.stem + "_sub.mp4")
DUBBED_VIDEO_PATH = OUTPUT_DIR / ("04_" + VIDEO_PATH.stem + "_dubbed.mp4")
SUB_DUBBED_VIDEO_PATH = OUTPUT_DIR / ("05_" + VIDEO_PATH.stem + "_subdubbed.mp4")
LOG_PATH          = OUTPUT_DIR / "ffmpeg_pipeline.log"


## 第一节 / Section 1: 环境初始化

- 导入所需库：`os`、`re`、`asyncio`、`subprocess`、`logging`、`pathlib`
- 配置日志：同时输出到控制台和文件（`ffmpeg_pipeline.log`）
- 打印配置摘要供运行前确认


In [ ]:
import os
import re
import asyncio
import subprocess
import logging
from pathlib import Path

# ---------- 日志配置 ----------
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    level=LOG_LEVEL,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(LOG_PATH, encoding="utf-8"),
        logging.StreamHandler(),
    ],
)
logger = logging.getLogger(__name__)

# ---------- 配置摘要 ----------
logger.info("=" * 60)
logger.info("【Cell 3: 环境初始化】开始")
logger.info("=" * 60)
logger.info(f"视频路径        : {VIDEO_PATH}")
logger.info(f"输出目录        : {OUTPUT_DIR}")
logger.info(f"Whisper 模型    : {WHISPER_MODEL}")
logger.info(f"转录语言        : {WHISPER_LANGUAGE}")
logger.info(f"FP16 模式       : {WHISPER_FP16}")
logger.info(f"TTS 引擎        : {TTS_ENGINE}")
logger.info(f"TTS 声音        : {TTS_VOICE}")
if TTS_ENGINE == "f5":
    logger.info(f"参考音频        : {VOICE_CLONE_REF_AUDIO}")
    logger.info(f"参考文本        : {VOICE_CLONE_REF_TEXT[:50]}...")
logger.info(f"生成字幕视频    : {GENERATE_SUBTITLE_VIDEO}")
logger.info(f"SRT 路径        : {SRT_PATH}")
logger.info(f"配音 WAV 路径   : {DUB_WAV_PATH}")
logger.info(f"最终视频路径    : {DUBBED_VIDEO_PATH}")
logger.info(f"日志文件        : {LOG_PATH}")
logger.info("=" * 60)
logger.info("【Cell 3: 环境初始化】完成")
logger.info("=" * 60)


## 第二节 / Section 2: 语音转录（Whisper ASR）

使用 [OpenAI Whisper](https://github.com/openai/whisper) 对视频文件进行语音识别。

| 参数 | 含义 |
|------|------|
| `WHISPER_MODEL` | 模型大小，越大越准但越慢（`turbo` 为推荐均衡选项） |
| `WHISPER_LANGUAGE` | 指定语言可提高准确率（`"zh"` = 中文，`"en"` = 英文） |
| `WHISPER_FP16` | GPU 下开启可加速推理，CPU 环境设为 `False` |

**输出变量：** `result`（Whisper 转录结果字典，供后续 Cell 使用）


In [ ]:
import whisper

logger.info("=" * 60)
logger.info("【Cell 5: 语音转录】开始")
logger.info("=" * 60)

try:
    logger.info(f"加载 Whisper 模型: {WHISPER_MODEL}")
    model = whisper.load_model(WHISPER_MODEL)

    logger.info(f"开始转录: {VIDEO_PATH}")
    result = model.transcribe(
        str(VIDEO_PATH),
        language=WHISPER_LANGUAGE,
        fp16=WHISPER_FP16,
    )

    text_preview = result["text"][:200]
    logger.info(f"转录完成，文本预览（前 200 字符）:\n{text_preview}")
    logger.info(f"共识别 {len(result['segments'])} 个片段")
except Exception as e:
    logger.error(f"语音转录失败: {e}")
    raise

logger.info("=" * 60)
logger.info("【Cell 5: 语音转录】完成")
logger.info("=" * 60)


## 第三节 / Section 3: 生成 SRT 字幕

将 Whisper 转录的时间戳片段（`segments`）格式化为标准 SRT 字幕文件。

SRT 格式示例：
```
1
00:00:01,234 --> 00:00:05,678
字幕文本内容

2
...
```

**输出文件：** `SRT_PATH`


In [ ]:
logger.info("=" * 60)
logger.info("【Cell 7: 生成 SRT 字幕】开始")
logger.info("=" * 60)


def format_time(t: float) -> str:
    """将秒数转换为 SRT 时间格式 HH:MM:SS,mmm"""
    hrs  = int(t // 3600)
    mins = int((t % 3600) // 60)
    secs = int(t % 60)
    ms   = int((t - int(t)) * 1000)
    return f"{hrs:02}:{mins:02}:{secs:02},{ms:03}"


try:
    segments = result["segments"]

    with open(SRT_PATH, "w", encoding="utf-8") as f:
        for i, seg in enumerate(segments, start=1):
            f.write(f"{i}\n")
            f.write(f"{format_time(seg['start'])} --> {format_time(seg['end'])}\n")
            f.write(seg["text"].strip() + "\n\n")

    srt_size = SRT_PATH.stat().st_size
    logger.info(f"SRT 字幕已写出: {SRT_PATH}")
    logger.info(f"字幕条数: {len(segments)}")
    logger.info(f"文件大小: {srt_size} 字节")
except Exception as e:
    logger.error(f"生成 SRT 字幕失败: {e}")
    raise

logger.info("=" * 60)
logger.info("【Cell 7: 生成 SRT 字幕】完成")
logger.info("=" * 60)


## 第四节 / Section 4: 嵌入字幕到视频（可选）

> 由 `GENERATE_SUBTITLE_VIDEO` 开关控制，默认开启。

使用 FFmpeg 将 SRT 字幕软嵌入视频（`mov_text` 格式），生成 `*_sub.mp4`。

- **软字幕**：不烧录到画面，可在播放器中开关
- 视频流和音频流均使用 `-c copy`，不重新编码，速度极快

**输出文件：** `SUB_VIDEO_PATH`


In [ ]:
logger.info("=" * 60)
logger.info("【Cell 9: 嵌入字幕到视频】开始")
logger.info("=" * 60)

if not GENERATE_SUBTITLE_VIDEO:
    logger.info("GENERATE_SUBTITLE_VIDEO=False，跳过此步骤")
else:
    try:
        cmd = [
            "ffmpeg", "-y",
            "-i", str(VIDEO_PATH),
            "-i", str(SRT_PATH),
            "-c:v", "copy",
            "-c:a", "copy",
            "-c:s", "mov_text",
            str(SUB_VIDEO_PATH),
        ]
        logger.info(f"执行命令: {' '.join(cmd)}")
        ret = subprocess.run(cmd, capture_output=True)
        logger.info(f"ffmpeg 返回码: {ret.returncode}")
        if ret.returncode != 0:
            stderr_txt = ret.stderr.decode("utf-8", errors="replace")
            logger.error(f"ffmpeg stderr:\n{stderr_txt}")
            raise RuntimeError(f"ffmpeg 执行失败，返回码: {ret.returncode}")
        logger.info(f"字幕视频已生成: {SUB_VIDEO_PATH}")
    except Exception as e:
        logger.error(f"嵌入字幕失败: {e}")
        raise

logger.info("=" * 60)
logger.info("【Cell 9: 嵌入字幕到视频】完成")
logger.info("=" * 60)


## 第五节 / Section 5: TTS 配音音轨生成

对每条 SRT 字幕：

1. 使用 `edge-tts` 合成语音（MP3）
2. 测量合成音频时长与字幕时长的比例
3. 若音频过长，使用 FFmpeg `atempo` 滤镜加速压缩（支持链式 > 2.0x）
4. 用 `pydub` 拼接所有片段，补充静音对齐时间轴
5. 导出 48kHz 立体声 WAV

**输出文件：** `DUB_WAV_PATH`


In [ ]:
import srt
from pydub import AudioSegment

logger.info("=" * 60)
logger.info("【Cell 11: TTS 配音音轨生成】开始")
logger.info("=" * 60)


def sanitize_text(t: str) -> str:
    t = t.strip()
    t = re.sub(r"\s+", " ", t)
    return t


async def generate_tts_edge(text: str, out_path: Path):
    import edge_tts
    communicate = edge_tts.Communicate(text=text, voice=TTS_VOICE)
    await communicate.save(str(out_path))


def generate_tts_f5(text: str, out_path: Path):
    wav, sr, _ = f5_model.infer(
        ref_file=str(VOICE_CLONE_REF_AUDIO),
        ref_text=VOICE_CLONE_REF_TEXT,
        gen_text=text,
    )
    sf.write(str(out_path), wav, sr)


def ffmpeg_time_stretch(in_audio: Path, out_wav: Path, speed: float):
    """
    speed > 1.0 means faster (shorter audio).
    atempo supports 0.5~2.0; chain filters if outside that range.
    """
    filters = []
    s = speed
    while s > 2.0:
        filters.append("atempo=2.0")
        s /= 2.0
    while s < 0.5:
        filters.append("atempo=0.5")
        s *= 2.0
    filters.append(f"atempo={s:.6f}")
    af = ",".join(filters)
    cmd = ["ffmpeg", "-y", "-i", str(in_audio), "-af", af, str(out_wav)]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


try:
    TTS_TMP_DIR.mkdir(exist_ok=True)

    # 加载 TTS 模型（F5-TTS 需要，edge-tts 无需）
    if TTS_ENGINE == "f5":
        import gc
        import torch

        # 释放 Whisper 模型显存，为 F5-TTS 腾出空间
        try:
            del model
            logger.info("已删除 Whisper 模型对象")
        except NameError:
            pass
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            free_mb = torch.cuda.mem_get_info()[0] // (1024 * 1024)
            logger.info(f"CUDA 缓存已清空，当前可用显存: {free_mb} MiB")

        from f5_tts.api import F5TTS
        import soundfile as sf
        logger.info("加载 F5-TTS 模型（首次运行需下载，约 1.2GB）...")
        f5_model = F5TTS()
        logger.info("F5-TTS 模型加载完成")

    raw_ext = "wav" if TTS_ENGINE == "f5" else "mp3"

    subs = list(srt.parse(SRT_PATH.read_text(encoding="utf-8")))
    full = AudioSegment.silent(duration=0)

    for i, sub in enumerate(subs, start=1):
        text = sanitize_text(sub.content)
        if not text:
            logger.warning(f"第 {i} 条字幕文本为空，跳过")
            continue

        start_ms  = int(sub.start.total_seconds() * 1000)
        end_ms    = int(sub.end.total_seconds() * 1000)
        target_ms = max(1, end_ms - start_ms)

        raw_file  = TTS_TMP_DIR / f"{i:05d}_raw.{raw_ext}"
        fixed_wav = TTS_TMP_DIR / f"{i:05d}_fixed.wav"

        # 1) TTS 合成
        try:
            if TTS_ENGINE == "edge":
                await generate_tts_edge(text, raw_file)
            elif TTS_ENGINE == "f5":
                generate_tts_f5(text, raw_file)   # 同步，不需要 await
            else:
                raise ValueError(f"不支持的 TTS_ENGINE: {TTS_ENGINE!r}，请设置为 'edge' 或 'f5'")
        except Exception as e:
            logger.error(f"第 {i} 条字幕 TTS 合成失败: {e}")
            raise

        seg    = AudioSegment.from_file(raw_file)   # pydub 根据扩展名自动识别格式
        seg_ms = len(seg)
        speed  = 1.0

        # 2) 对齐时长（音频 > 目标时长时加速压缩）
        if seg_ms > target_ms:
            speed = seg_ms / target_ms
            ffmpeg_time_stretch(raw_file, fixed_wav, speed)
            seg    = AudioSegment.from_wav(fixed_wav)
            seg_ms = len(seg)

        if seg_ms < target_ms:
            seg = seg + AudioSegment.silent(duration=(target_ms - seg_ms))
        else:
            seg = seg[:target_ms]

        # 3) 补齐起始静音，追加段落
        if len(full) < start_ms:
            full = full + AudioSegment.silent(duration=(start_ms - len(full)))
        elif len(full) > start_ms:
            # 字幕时间戳重叠（Whisper 末尾常见），截断到正确起始位置
            logger.warning(
                f"字幕 {i:03d}: 时间戳重叠（full={len(full)}ms > start={start_ms}ms），截断"
            )
            full = full[:start_ms]
        full = full + seg

        logger.info(
            f"字幕 {i:03d}/{len(subs)}: '{text[:30]}' | "
            f"速度比={speed:.3f} | target={target_ms}ms"
        )

    # 导出整条配音音轨（48kHz 立体声）
    full = full.set_frame_rate(48000).set_channels(2)
    full.export(DUB_WAV_PATH, format="wav")
    logger.info(f"配音音轨已导出: {DUB_WAV_PATH}")
    logger.info(f"总时长: {len(full) / 1000:.2f} 秒")
except Exception as e:
    logger.error(f"TTS 配音生成失败: {e}")
    raise

logger.info("=" * 60)
logger.info("【Cell 11: TTS 配音音轨生成】完成")
logger.info("=" * 60)


## 第六节 / Section 6: 替换音频生成最终视频

使用 FFmpeg 将原视频的视频流与配音 WAV 音轨合并：

- `-c:v copy`：视频流直接复制，**不重新编码**，保持原画质和速度
- `-map 0:v:0`：取原视频的第一路视频流
- `-map 1:a:0`：取配音 WAV 的第一路音频流
- `-shortest`：以较短的流为准截断输出，防止长度不一致

**输出文件：** `DUBBED_VIDEO_PATH`


In [ ]:
logger.info("=" * 60)
logger.info("【Cell 13: 替换音频生成最终视频】开始")
logger.info("=" * 60)

try:
    cmd = [
        "ffmpeg", "-y",
        "-i", str(VIDEO_PATH),
        "-i", str(DUB_WAV_PATH),
        "-c:v", "copy",
        "-map", "0:v:0",
        "-map", "1:a:0",
        "-shortest",
        str(DUBBED_VIDEO_PATH),
    ]
    logger.info(f"执行命令: {' '.join(cmd)}")
    ret = subprocess.run(cmd, capture_output=True)
    logger.info(f"ffmpeg 返回码: {ret.returncode}")
    if ret.returncode != 0:
        stderr_txt = ret.stderr.decode("utf-8", errors="replace")
        logger.error(f"ffmpeg stderr:\n{stderr_txt}")
        raise RuntimeError(f"ffmpeg 执行失败，返回码: {ret.returncode}")

    out_size = DUBBED_VIDEO_PATH.stat().st_size / (1024 * 1024)
    logger.info(f"最终配音视频已生成: {DUBBED_VIDEO_PATH}")
    logger.info(f"文件大小: {out_size:.2f} MB")
except Exception as e:
    logger.error(f"替换音频失败: {e}")
    raise

logger.info("=" * 60)
logger.info("【Cell 13: 替换音频生成最终视频】完成")
logger.info("=" * 60)


## 第七节 / Section 7: 嵌入字幕到视频（可选）

> 由 `GENERATE_SUBTITLE_VIDEO` 开关控制，默认开启。

使用 FFmpeg 将 SRT 字幕软嵌入视频（`mov_text` 格式），生成 `*_subdubbed.mp4`。

- **软字幕**：不烧录到画面，可在播放器中开关
- 视频流和音频流均使用 `-c copy`，不重新编码，速度极快

**输出文件：** `SUB_DUBBED_VIDEO_PATH`


In [ ]:
logger.info("=" * 60)
logger.info("【Cell 9: 嵌入字幕到视频】开始")
logger.info("=" * 60)

if not GENERATE_SUBTITLE_VIDEO:
    logger.info("GENERATE_SUBTITLE_VIDEO=False，跳过此步骤")
else:
    try:
        cmd = [
            "ffmpeg", "-y",
            "-i", str(DUBBED_VIDEO_PATH),
            "-i", str(SRT_PATH),
            "-c:v", "copy",
            "-c:a", "copy",
            "-c:s", "mov_text",
            str(SUB_DUBBED_VIDEO_PATH),
        ]
        logger.info(f"执行命令: {' '.join(cmd)}")
        ret = subprocess.run(cmd, capture_output=True)
        logger.info(f"ffmpeg 返回码: {ret.returncode}")
        if ret.returncode != 0:
            stderr_txt = ret.stderr.decode("utf-8", errors="replace")
            logger.error(f"ffmpeg stderr:\n{stderr_txt}")
            raise RuntimeError(f"ffmpeg 执行失败，返回码: {ret.returncode}")
        logger.info(f"字幕视频已生成: {SUB_DUBBED_VIDEO_PATH}")
    except Exception as e:
        logger.error(f"嵌入字幕失败: {e}")
        raise

logger.info("=" * 60)
logger.info("【Cell 9: 嵌入字幕到视频】完成")
logger.info("=" * 60)


## 完成 / Done 🎉

### 输出文件汇总

| 文件 | 说明 |
|------|------|
| `*.srt` | SRT 字幕文件 |
| `*_sub.mp4` | 带字幕视频（可选，由 `GENERATE_SUBTITLE_VIDEO` 控制） |
| `*_dub.wav` | 配音音轨 WAV |
| `*_dubbed.mp4` | 配音视频 |
| `*_subdubbed.mp4` | **最终配音+字幕视频**（主要输出） |
| `ffmpeg_pipeline.log` | 完整运行日志 |

---

> 如需更换声音，在 **Cell 1** 修改 `TTS_VOICE` 后，从 **Cell 11** 重新运行即可。
